In [ ]:
#With this set of codes, the full credit goes to Adam and his nice introduction video for Python with Dash and Plotly
#The link to the video is: https://www.youtube.com/watch?v=hSPmj7mK6ng
#You can also find this set of code on Adam's GitHub page: https://bit.ly/3JldJeH
#Adam has many more tutorial videos for Python with Dash and Plotly. You can follow his great YouTube channel at: https://bit.ly/3Jiyw2F
#I did not fork this set of code from Adam's GitHub page because I wanted to code it by hand, line-by-line, after watching his tutorial video.
#I think that it's a great opportunity to learn and understand how to get Dash and Plotly to work with Python.

#NOTE: For the code to fully run, make sure that debug=False on the last line.


import pandas as pd
import plotly.express as px
import plotly.graph_objects as go
from dash import Dash, dcc, html, Input, Output

app = dash.Dash(__name__)

import os #working with directories

#-------------------------------------------------------------
#change directory
app = Dash(__name__)

wd = "C:/Users/Acer/Desktop/Python Interactive Dashboard with Plotly/1_bees Choroplethmap data"
os.chdir(wd)
print(os.getcwd())

#import and clean data

df = pd.read_csv("intro_bees.csv")
#df.head()

df = df.groupby(['State', 'ANSI', 'Affected by', 'Year', 'state_code'])[['Pct of Colonies Impacted']].mean()
df.reset_index(inplace=True)
print(df[:5])

#App layout

app.layout = html.Div([
    
    html.H1("HTML Dahsboard Application with Dash", style = {'text-align':'center'}),
    
    dcc.Dropdown(id = "slct_year",
                options=[
                    {"label":"2015", "value":2015},
                    {"label":"2016", "value":2016},
                    {"label":"2017", "value":2017},
                    {"label":"2018", "value":2018}],
                multi = False,
                value = 2015,
                style = {"width":"40%"}
                ),
    html.Div(id = 'output_container', children = []),
    html.Br(),
    
    dcc.Graph(id = 'my_bee_map', figure = {})
])

#connect Plotly graphs to Dash components

@app.callback(
    [Output(component_id='output_container', component_property='children'),
     Output(component_id='my_bee_map', component_property='figure')],
    [Input(component_id='slct_year', component_property='value')]
)

def update_graph(option_slctd):
    print(option_slctd)
    print(type(option_slctd))
    
    container = "The year chosen by user was: {}".format(option_slctd)
    
    dff = df.copy()
    dff = dff[dff["Year"] == option_slctd]
    dff = dff[dff["Affected by"] == "Varroa_mites"]
    
    #Plotly Express
    
    fig = px.choropleth(
        data_frame = dff,
        locationmode = 'USA-states',
        locations = 'state_code',
        scope = 'usa',
        color='Pct of Colonies Impacted',
        hover_data = ['State', 'Pct of Colonies Impacted'],
        color_continuous_scale = px.colors.sequential.YlOrRd,
        labels = {'Pct of Colonies Impacted':'% of Bees Colonies'},
        template = 'plotly_dark'
    )
       
    return container, fig

# ------------------------------------------------------------------------------
if __name__ == '__main__':
    app.run_server(debug=False)